In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import math
import numpy as np
import numpy.matlib
import random
import os
import pathlib
import time
from datetime import date

# Set seed
np.random.seed(42)
random.seed(42)

# Classes


In [3]:
class Individual:
  """
  A class that represents an individual solution.

  Attributes:
    chromosome (1D list): The chromosome of the individual in [0,1] interval.
    weights (2D list): The weights of the individual in [-1,1] interval.
    fitness (float): The fitness of the individual.
  """

  #TODO: Add bias as an attribute of the Individual and tune it as well as the weights and chromosome
  def __init__(self, chromosome: np.ndarray, weights: np.ndarray, bias: np.ndarray,  fitness:float = math.inf):
    self._chromosome = chromosome
    self._weights = weights
    self._bias = bias
    
    if fitness < 0.0:
      raise ValueError('The fitness of an individual cannot be negative.')
    else:
      self._fitness = fitness
  
  def __str__(self):
    return f"""
    Individual:
        Feature-selection: {self._chromosome}
        Weights: {self._weights}
        Bias: {self._bias}
        Fitness: {self._fitness}
    """
  @property
  def chromosome(self):
    return self._chromosome
  
  @property
  def weights(self):
    return self._weights
  
  @property
  def bias(self):
    return self._bias

  @property
  def fitness(self):
    return self._fitness

  @chromosome.setter
  def chromosome(self, chromosome):
    self._chromosome = chromosome

  @weights.setter
  def weights(self, weights):
    self._weights = weights

  @bias.setter
  def bias(self, bias):
    self._bias = bias

  @fitness.setter
  def fitness(self,f):
    self._fitness = f

  @staticmethod
  def create_individual(K:int = 0, D:int = 0):
    """
    Generates a random individual.

    Args:
      D (int): The number of hidden neurons.
      K (int): The number of input features.

    Returns:
      Individual: A random individual.
    """
    chromosome= np.random.randint(low= 0, high= 1 + 1, size = K)
    weights= np.random.uniform(low= -1, high= 1, size=(K,D))
    bias= np.random.uniform(low= -1, high= 1, size=(1,D))
    return Individual(chromosome, weights, bias)

In [5]:
class Particle(Individual):
  """
  A class that represents an individual solution in Swarm-based problems.

  Attributes:
  -----------
    (same as Individual class), and additional attributes:

    best_fitness (float): The best fitness of the individual.
    best_weights_position (numpy.ndarray): The best weight position of the individual.
    best_chromosome_position (numpy.ndarray): The best chromosome position of the individual.
    weights_velocity (numpy.ndarray): The velocity of the weights of the individual.
    chromosome_velocity (numpy.ndarray): The velocity of the chromosome of the individual.  
  """
  
  def __init__(self, chromosome: np.ndarray, weights: np.ndarray, bias: np.ndarray, fitness:float = math.inf):
    super().__init__(chromosome, weights, bias, fitness)
    
    self._best_fitness = fitness # At the beginning, the best fitness is the input fitness
    self._best_weights_position = weights # By default, the best weight position is the starting weight position.
    self._best_bias_position = bias # By default, the best bias position is the starting bias position.
    self._best_chromosome_position = chromosome # By default, the best chromosome position is the starting chromosome position.
    self._weights_velocity = np.zeros(weights.shape) # All swarm's weights velocities are set to 0 at the start
    self._bias_velocity = np.zeros(bias.shape) # All swarm's bias velocities are set to 0 at the start
    self._chromosome_velocity = np.zeros(chromosome.shape) # All swarm's chromosome velocities are set to 0 at the start

  def __str__(self):
    return f"""
    Particle:
        Fitness: {self.fitness}
        Actual weights velocity: {self._weights_velocity}
        Actual bias velocity: {self._bias_velocity}
        Actual feature-selection velocity: {self._chromosome_velocity}
        Best fitness: {self._best_fitness}
    """

  @property
  def best_fitness(self):
    return self._best_fitness

  @property
  def best_weights_position(self):
    return self._best_weights_position
  
  @property
  def best_bias_position(self):
    return self._best_bias_position

  @property
  def best_chromosome_position(self):
    return self._best_chromosome_position
  
  @property
  def weights_velocity(self):
    return self._weights_velocity

  @property
  def bias_velocity(self):
    return self._bias_velocity

  @property
  def chromosome_velocity(self):
    return self._chromosome_velocity

  @best_fitness.setter
  def best_fitness(self,f):
    self._best_fitness = f

  @best_weights_position.setter
  def best_weights_position(self,w):
    self._best_weights_position = w

  @best_bias_position.setter
  def best_bias_position(self,b):
    self._best_bias_position = b

  @best_chromosome_position.setter
  def best_chromosome_position(self,c):
    self._best_chromosome_position = c

  @weights_velocity.setter
  def weights_velocity(self,w):
    self._weights_velocity = w

  @bias_velocity.setter
  def bias_velocity(self,b):
    self._bias_velocity = b
    
  @chromosome_velocity.setter
  def chromosome_velocity(self,c):
    self._chromosome_velocity = c

  @staticmethod
  def create_particle(K:int = 0, D:int = 0):
    """
    Generates a random particle.

    Args:
      D (int): The number of hidden neurons.
      K (int): The number of input features.

    Returns:
      Particle: A random particle.
    """
    chromosome= np.random.randint(low= 0, high= 1 + 1, size = K)
    weights= np.random.uniform(low= -1, high= 1, size=(K,D))
    bias= np.random.uniform(low= -1, high= 1, size=(1,D))
    return Particle(chromosome, weights, bias)

In [6]:
class Population:
  """
  A class that represents a population of individual solutions.

  Attributes:
  -----------
    size (int): The size of the population.
    gene_list (list): The list of individuals solutions.
    best_gene (Individual): The best individual of the population.
  
  Methods:
  -----------
    insert_best_gene(Individual): Inserts the best individual of the population into the gene_list.
    add_gene_to_list_at_index(Individual): Adds an individual to the gene_list.
  """

  def __init__(self, size:int, K:int = 0, D:int = 0, is_empty:bool = False, is_swarm:bool = False):
    self._size = size
    self._best_gene = None

    if is_empty == True:
      self._genes_list = np.empty(size, dtype=Individual)
    else:
      if is_swarm == True:
        self._genes_list = np.array([Particle.create_particle(K,D) for _ in range(size)]) 
      else:
        self._genes_list = np.array([Individual.create_individual(K,D) for _ in range(size)]) 
  
  def __str__(self):
    return f"""
    Population:
        Size: {self._size}
        Gene list: {self._genes_list}
        Best gene: {self._best_gene}
    """
  @property
  def size(self):
    return self._size
  
  @property
  def genes_list(self):
    return self._genes_list

  @property
  def best_gene(self):
    return self._best_gene

  @genes_list.setter
  def genes_list(self, genes_list):
    self._genes_list = genes_list
    
  @best_gene.setter
  def best_gene(self,gen):  
    self._best_gene = gen
  
  @staticmethod
  def create_population(size:int, K:int = 0, D:int = 0, is_empty:bool = False, is_swarm:bool = False):
    """
    Generates a random population.

    Args:
      size (int): The size of the population.
      D (int): The number of hidden neurons.
      K (int): The number of input features.
      is_empty (bool): If True, the population will be empty.

    Returns:
      Population: A random population.
    """
    return Population(size, K, D, is_empty, is_swarm)

  def add_gene_to_list_at_index(self,gen, index:int):
    self._genes_list[index] = gen

  def insert_best_gene(self,gen):
    self.best_gene = gen
    self._genes_list[0] = gen  # Insert the best gene at the beginning of the list

In [7]:
class Swarm(Population):
  """
  A class that represents a population of solutions in Swarm-based problems.

  Attributes:
  -----------
    (same as Population), and additional attributes:

    global_best_fitness (float): The best fitness of the population.
    global_best_weights (numpy.ndarray): The best weight position of the population.
    global_best_chromosome (numpy.ndarray): The best chromosome position of the population.    
  
  Methods:
  -----------

  """

  #FIXME: Cuidado, puede que en vez de Nones tenga que instanciarlos porque en la primera iteración?¿?
  def __init__(self, size:int, K:int = 0, D:int = 0, is_empty:bool = False):
    super().__init__(size, K, D, is_empty, is_swarm=True)
    self._global_best_fitness = math.inf
    self._global_best_weights = None
    self._global_best_chromosome = None
    self._global_best_bias = None

  def __str__(self):
    return f"""
    Swarm:
        Size: {self._size}
        Particles list: {self.genes_list}
        Best particle: {self.best_gene}
        Global best fitness: {self._global_best_fitness}
        Global best weights: {self._global_best_weights}
        Global best bias: {self._global_best_bias}
        Global best feature-selection: {self._global_best_chromosome}
    """
  @property
  def global_best_fitness(self):
    return self._global_best_fitness

  @property
  def global_best_weights(self):
    return self._global_best_weights

  @property
  def global_best_chromosome(self):
    return self._global_best_chromosome

  @property
  def global_best_bias(self):
    return self._global_best_bias

  @global_best_fitness.setter
  def global_best_fitness(self,f):
    self._global_best_fitness = f
  
  @global_best_weights.setter
  def global_best_weights(self,w):
    self._global_best_weights = w
  
  @global_best_chromosome.setter
  def global_best_chromosome(self,c):
    self._global_best_chromosome = c

  @global_best_bias.setter
  def global_best_bias(self,b):
    self._global_best_bias = b

  @classmethod
  def create_swarm(size:int, K:int = 0, D:int = 0, is_empty:bool = False):
    """
    Generates a random swarm.

    Args:
      size (int): The size of the swarm.
      D (int): The number of hidden neurons.
      K (int): The number of input features.
      is_empty (bool): If True, the swarm will be empty.

    Returns:
      Swarm: A random swarm.
    """
    return Swarm(size, K, D, is_empty)

In [8]:
class Reef:
  """
  A class that represents a population of solutions in Evolutionary-based problems.

  Attributes:
  ----------- 
    size (int): The size of the population.
    free_occupied_rate (float): The rate of occupied individuals in the reef.
    genes_list (list): The list of individuals solutions.
    best_gene (Individual): The best individual of the population.
    
  Methods:
  -----------
  """

  # TODO: Add array with indexes of corals sorted by fitness (from best to worst)
  def __init__(self, size:int, rate:float, K:int, D:int):
    self._size = size
    self._free_occupied_rate = rate
    self._best_coral = None
    self._corals_list = np.full(shape = [size], fill_value = None) # Empty reef

    occupiedHoles = int(size * rate) # Partially occupy the reef
    self._corals_list[:occupiedHoles] = np.array([Individual.create_individual(K,D) for _ in range(occupiedHoles)]) # Fill the reef with individuals

  def __str__(self):
    return f"""
    Reef:
        Size: {self._size}
        Free-occupied rate: {self._free_occupied_rate}
        Corals list: {self._corals_list}
        Best coral: {self._best_coral}
    """
  
  @property
  def size(self):
    return self._size

  @property
  def free_occupied_rate(self):
    return self._free_occupied_rate
  
  @property
  def corals_list(self):
    return self._corals_list

  @property
  def best_coral(self):
    return self._best_coral
  
  @corals_list.setter
  def corals_list(self,corals_list):
    self._corals_list = corals_list

  @best_coral.setter
  def best_coral(self,coral):
    self._best_coral = coral
  
  @classmethod
  def create_reef(size:int, rate:float, K:int, D:int):
    """
    Generates a reef.

    Args:
      size (int): The size of the reef.
      rate (float): The rate of occupied individuals in the reef.
      D (int): The number of hidden neurons.
      K (int): The number of input features.

    Returns:
      Reef: A reef.
    """
    return Reef(size, rate, K, D)

  @classmethod
  def insert_new_larvae_in_hole(self, larvae: Individual, hole_index:int):
    self.corals_list[hole_index] = larvae

  @classmethod
  def remove_coral_from_hole(self, hole_index:int):
    self.corals_list[hole_index] = None
  
  @classmethod
  def sort_by_fitness(self):
    # Sort corals_list by larvaes fitness in descending order, and if there are None values, put them at the end of the list
    self.corals_list = sorted(self.corals_list, key=lambda x: x.fitness if x is not None else math.inf)


# METHODS


In [9]:
def sum_matrixes(mat1:np.ndarray, mat2:np.ndarray):
  return np.add(mat1, mat2)

def subtract_matrixes(mat1:np.ndarray, mat2:np.ndarray):
  return np.subtract(mat1, mat2)

def dot_multiply(mat1:np.ndarray, mat2:np.ndarray):
  return np.multiply(mat1, mat2)

def multiply_matrixes(mat1:np.ndarray, mat2:np.ndarray):
  return np.matmul(mat1, mat2)

def dot_divide(mat1:np.ndarray, mat2:np.ndarray):
  return np.divide(mat1, mat2)

In [10]:
array = np.array([[1,2,3,4,5,6,7,8,9,10], [2,3,4,5,6,7,8,9,10,11]])

print(array.shape)
print(len(array))

(2, 10)
2


In [11]:
def internal_reproduction(gen: Individual):
  # Mutate chromosome
  mutate_chromosome_point = np.random.randint(0, gen.chromosome.shape[0]) # Choose a random feature to mutate
  gen.chromosome[mutate_chromosome_point] = 1 - gen.chromosome[mutate_chromosome_point] # Activate or deactivate the feature
  
  # Mutate weights
  total_rows = gen.weights.shape[0] 
  total_columns = gen.weights.shape[1] 
  mutate_column = np.random.randint(0, total_columns) # Choose a random column to mutate
  
  gen.weights[:,mutate_column] = np.random.uniform(-1,1,total_rows) # Mutate the column

  # Mutate bias
  mutate_bias_point = np.random.randint(0, gen.bias.shape[1]) # Choose a random column to mutate
  gen.bias[0,mutate_bias_point] = np.random.uniform(-1,1) # Mutate the column

  print(f"Mutation feature {mutate_chromosome_point}, and bias {mutate_bias_point} of bias matrix")
  print(f"After mutation: {gen}")

In [12]:
def external_reproduction(father:Individual, mother:Individual):
  crossover_point = np.random.randint(0, len(father.chromosome))
  crossover_column = random.randint(0, len(father.weights[0]))
  
  # Offsprings are created by combining father and mother chromosomes and weights
  offspring1 = Individual( chromosome = np.concatenate((father.chromosome[:crossover_point], mother.chromosome[crossover_point:])), 
                           weights = np.concatenate((father.weights[:,:crossover_column], mother.weights[:,crossover_column:]), axis=1),
                           bias = np.concatenate((father.bias[:,:crossover_column], mother.bias[:,crossover_column:]), axis=1) )
  
  offspring2 = Individual( chromosome = np.concatenate((mother.chromosome[:crossover_point], father.chromosome[crossover_point:])), 
                           weights = np.concatenate((mother.weights[:,:crossover_column], father.weights[:,crossover_column:]), axis=1) ) 
                       
  return [offspring1,offspring2] 

In [13]:
def roulette_wheel_selection(population:Population): 
  # Create an empty population for the chosen individuals of the Roulette Wheel
  roulettePopulation = create_population(population.size, is_empty=True)
  roulettePopulation.insert_best_gene(population.best_gene) # Make sure we don't lose our best gene in the roulette 

  # Total population fitness (S)
  S = np.sum([individual.fitness for individual in population.genes_list])

  # Population chromosomes' relative probabilities
  rel_prob = [individual.fitness/S for individual in population.genes_list]

  for idx in range(1, roulettePopulation.size): # -1 because we already inserted the best gene
    r = np.random.uniform() 
    # Find the first index for which q_i < r
    for index,individual in enumerate(population.genes_list): 

      r -= rel_prob[index] 
      if r < 0:
        roulettePopulation.add_gene_to_list_at_index(individual, idx)
        break
    # end choose rouletted individual loop
  # end roulette population loop
  return roulettePopulation  

In [14]:
''' This functions reproduces a population.
    It crossover the parents (external reproduction) and mutate the offspring (internal reproduction).
    Returns the input population, updated with the crossover and mutated childs '''

#FIXME: PASS PROBABILITES AS PARAMETERS
def reproduce_population(population: Population):
  childs = np.empty(shape=0, dtype=Individual) # list to store future offsprings
  crossover_probability = np.random.uniform()
  mutation_probability = np.random.uniform()
  gene_list_size = len(population.genes_list)

  for index,individual in enumerate(population.genes_list):
    # Generate a random mother if the crossover probability is met
    if(np.random.random() >= crossover_probability):
      random_mother_index = np.random.randint(0,gene_list_size) # Choose a random mother

      while(random_mother_index == index): # If the mother is the same as the father, choose another one
        random_mother_index = np.random.randint(0,gene_list_size)
      #end while
      
      # Crossover parents
      mother = population.genes_list[random_mother_index]
      offsprings = external_reproduction(individual, mother)

      # Mutate offsprings if probability is over the mutation probability
      rand_value = np.random.random()
      if(rand_value >= mutation_probability): 
        internal_reproduction(offsprings[0])

      if(rand_value >= mutation_probability):
        internal_reproduction(offsprings[1])

      # Add offsprings to the childs list
      childs = np.append(childs, offsprings)
  
  # Extend the population with the new childs
  population.genes_list = np.concatenate( (population.genes_list, childs) )
  return population

In [15]:
def update_velocities_and_positions_pso(swarm: Swarm, w:float, c1:float, c2:float):
  # FIXME: 
  # Velocities
  r1 = np.random.uniform()
  r2 = np.random.uniform()

  for index, particle in enumerate(swarm.genes_list): # For each particle in the swarm
    # Chromosome loop
    for k in range(len(particle.chromosome)):
      # Update chromosome velocity 
      particle.chromosome_velocity[k] = ( 
                                          (w * particle.chromosome_velocity[k]) +
                                          (c1*r1*(particle.best_chromosome_position[k] - particle.chromosome[k])) +
                                          (c2*r2*(swarm.global_best_chromosome[k] - particle.chromosome[k]))
                                        )

      # Prevent velociy from going out of bounds
      if(particle.chromosome_velocity[k] > 6):
        particle.chromosome_velocity[k] = 6
      elif(particle.chromosome_velocity[k] < -6):
        particle.chromosome_velocity[k] = -6

      velocityProbability = 2 / math.pi * math.atan((math.pi*0.5)*particle.chromosome_velocity[k]) #|2⁄𝜋 × arctan ((𝜋 2) × 𝑉𝑡+1

      # Update to next position
      if(np.random.uniform() < velocityProbability):
        particle.chromosome[k] = 1
      else:
        particle.chromosome[k] = 0
    # end chromosome loop

    # Weight loop
    for i in range(particle.weights.shape[0]):
      for j in range(particle.weights.shape[1]):

        particle.weights_velocity[i,j] = ( 
                                          (w * particle.weights_velocity[i,j]) +
                                          (c1*r1*(particle.best_weights_position[i,j] - particle.weights[i,j])) +
                                          (c2*r2*(swarm.global_best_weights[i,j] - particle.weights[i,j]))
                                        )
        # Update to next position
        particle.weights[i,j] += particle.weights_velocity[i,j]
    # end weight loop
  # end particle loop
  return swarm

In [16]:
def split_reef_candidates(candidates:list, fraction:float):
    number_of_candidates = len(candidates)
    split_point = int(number_of_candidates * fraction)

    # Split candidates in two lists
    broadcast_candidates = candidates[:split_point]
    brooding_candidates = candidates[split_point:]
    
    # If broadcast candidates are odd, add one more candidate to the list and remove it from the brooding candidates
    if len(broadcast_candidates) % 2 != 0:
        broadcast_candidates.append(candidates[split_point])
        brooding_candidates.remove(candidates[split_point])

    return [broadcast_candidates, brooding_candidates]

# ELM


In [17]:
# Computes the fitness for a given Individual using Extreme Learning Machine (ELM) model
def compute_individual_fitness(individual:Individual, D: int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
    # W
    individualWeights = dot_multiply(individual.weights, individual.chromosome[:, np.newaxis])

    # Feature-selection Bias given Gene's chromosome
    # FIXME: Will be replaced by TODO in the Individuals class
    Bias = np.random.uniform(low= 0, high= 1, size=[D,1]) 

    # Amplify the matrix to the size of Xtraining and Xtest
    BiasTrainingMatrix = np.matlib.repmat(Bias,1,trainingX.shape[0])
    BiasTestMatrix = np.matlib.repmat(Bias,1,testX.shape[0])

    # Transpose BiasMatrix
    BiasTrainingMatrix = np.transpose(BiasTrainingMatrix)
    BiasTestMatrix = np.transpose(BiasTestMatrix)

    # H (Sigmoide function) 
    activationTraining = multiply_matrixes(trainingX, individualWeights) + BiasTrainingMatrix
    activationTest = multiply_matrixes(testX, individualWeights) + BiasTestMatrix
    
    H_Training = dot_divide( 1, (1 + np.exp(-activationTraining)) )    
    H_Test = dot_divide( 1, (1 + np.exp(-activationTest)) )

    # Beta
    aux = multiply_matrixes( np.transpose(H_Training) , H_Training )
    delta = np.identity(aux.shape[0]) * 10e-3

    inverse = np.linalg.inv(dot_divide(np.identity(D) , C) + delta + aux)

    # Complete Formula: inv( (eye(D) ./ C) + delta + aux) * H_Training' * Ytraining;
    Beta = multiply_matrixes(inverse, np.transpose(H_Training))
    Beta = multiply_matrixes(Beta, trainingY)

    # Output
    Y_predicted = multiply_matrixes(H_Test, Beta)
    fitness = np.linalg.norm(Y_predicted - testY)

    return fitness

In [18]:
# Computes and udpates the fitnesses of a given Population 
def compute_population_fitness(population: Population, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
  for individual in population.genes_list:
    fitness_i = compute_individual_fitness(individual, D, C, trainingX, trainingY, testX, testY)
    individual.fitness = fitness_i # Update fitness

    # Update best gene in Population
    if population.best_gene is None:
      population.best_gene = individual
    else:
      if individual.fitness < population.best_gene.fitness:
        population.best_gene = individual
  # end for

In [19]:
# Computes and udpates the fitnesses of a given Swarm
def compute_swarm_fitness(swarm: Swarm, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
  for particle in swarm.genes_list:  
    fitness_i = compute_individual_fitness(particle, D, C, trainingX, trainingY, testX, testY) 
    particle.fitness = fitness_i

    # Update personal bests and global best
    if particle.fitness < particle.best_fitness:
      particle.best_fitness = particle.fitness
      
      if swarm.best_gene is None:
        swarm.best_gene = particle
      else:
        if particle.fitness < swarm.best_gene.fitness:
          swarm.best_gene = particle

  # Set Swarm's best chromosome and weights
  swarm.global_best_chromosome = swarm.best_gene.chromosome
  swarm.global_best_weights = swarm.best_gene.weights
  swarm.global_best_fitness = swarm.best_gene.fitness

In [20]:
# Computes and udpates the fitnesses of a given Reef
def compute_reef_fitness(reef: Reef, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
  for index,coral in enumerate(reef.corals_list):  
    if coral is None: # If the coral is empty, skip it
      continue

    fitness_i = compute_individual_fitness(coral, D, C, trainingX, trainingY, testX, testY) 
    coral.fitness = fitness_i

    # Update best gene in Reef
    if reef.best_coral is None:
      reef.best_coral = coral
    else:
      if coral.fitness < reef.best_coral.fitness:
        reef.best_coral = coral

In [21]:
# Computes and udpates the fitnesses of a given Pool list
def compute_pool_fitness(pool: list, D:int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
    for larvae in pool:  
        fitness_i = compute_individual_fitness(larvae, D, C, trainingX, trainingY, testX, testY) 
        larvae.fitness = fitness_i
    # end for   
    return pool

In [22]:
def train_model_and_output_results(best_weigths:np.ndarray, best_chromosome:np.ndarray, D: int, C:float, trainingX:np.ndarray, trainingY:np.ndarray, testX:np.ndarray, testY:np.ndarray):
    # Feature-selected weights
    weights = dot_multiply(best_weigths, best_chromosome[:, np.newaxis]) 

    # Bias
    Bias = np.random.uniform(low= 0, high= 1, size=[D,1])

    # Amplify the matrix to the size of Xtraining and Xtest
    BiasTrainingMatrix = np.matlib.repmat(Bias,1,trainingX.shape[0])
    BiasTestMatrix = np.matlib.repmat(Bias,1,testX.shape[0])

    # Transpose BiasMatrix
    BiasTrainingMatrix = np.transpose(BiasTrainingMatrix)
    BiasTestMatrix = np.transpose(BiasTestMatrix)

    # H (Sigmoide function) 
    activationTraining = multiply_matrixes(trainingX, weights) + BiasTrainingMatrix
    activationTest = multiply_matrixes(testX, weights) + BiasTestMatrix
    
    H_Training = dot_divide( 1, (1 + np.exp(-activationTraining)) )    
    H_Test = dot_divide( 1, (1 + np.exp(-activationTest)) )

    # Beta
    aux = multiply_matrixes( np.transpose(H_Training) , H_Training )
    delta = np.identity(aux.shape[0]) * 10e-3

    inverse = np.linalg.inv(dot_divide(np.identity(D) , C) + delta + aux)

    # Complete Formula: inv( (eye(D) ./ C) + delta + aux) * H_Training' * Ytraining;
    Beta = multiply_matrixes(inverse, np.transpose(H_Training))
    Beta = multiply_matrixes(Beta, trainingY)

    # Prediction
    predictedLabels = multiply_matrixes(H_Test, Beta)

    correctPrediction = 0
    for i in range(testY.shape[0]):
      if np.round(predictedLabels[i]) == testY[i]:
        correctPrediction +=1
    
    CCR = correctPrediction / testY.shape[0]
    return CCR * 100

# BIO-INSPIRED ALGORITHMS


In [23]:
''' This functions apply the Evolutionary-based Genetic Algorithm to a given population '''
def ga(population: Population, max_generations:int, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
  t = 0
  while t < max_generations:    
    # Reproduction
    reproduce_population(population)

    # Compute Fitness
    compute_population_fitness(population, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

    # Roulette Selection
    population = roulette_wheel_selection(population)

    # Continue iterating
    t += 1
  # end while 
  return population

In [24]:
''' This method apply the Swarm-based algorithm of PSO to a given swarm'''
def pso(swarm: Swarm, max_generations:int, w_max:float,  w_min:float, c1:float, c2:float, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
  t = 0
  while t < max_generations:
    # Linearly decrease the inertia weight
    w = w_max - (w_max - w_min) * t / max_generations

    # Evaluate Particles' Fitness and update Pb_i and Gb
    compute_swarm_fitness(swarm, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)
  
    # Update Velocity and Position of each particle
    update_velocities_and_positions_pso(swarm, w, c1, c2)

    # Increase step
    t += 1

    # Debug
    # print(f"Global best gene fitness {swarm.best_gene.fitness} - at generation {t}")
    # print(f"Global best feature selection {swarm.best_gene.chromosome}")

  # end while
  return swarm

In [25]:
def larvae_setting(pool:np.ndarray, reef:Reef, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
    # Compute pool fitness
    pool = compute_pool_fitness(pool, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

    # Try to settle the larvaes from the pool
    for larvae in pool:
      random_hole = np.random.randint(0, reef.size) # Random hole in the reef
      
      if reef.corals_list[random_hole] is None: # If the hole is empty, settle the larva
        reef.insert_new_larvae_in_hole(larvae, random_hole)
      elif reef.corals_list[random_hole] is not None and larvae.fitness < reef.corals_list[random_hole].fitness:
        reef.insert_new_larvae_in_hole(larvae, random_hole)
      
      # TODO:Implements larvae's attempts to settle in other holes
      
      #else: # If the hole is not empty and the larva is not better than the coral, the larva dies
        # print(f"Discarding larvae {larvae.fitness} in hole {random_hole} which has a coral with fitness {reef.corals_list[random_hole].fitness}")
    # end for
    return reef


In [26]:
def asexual_reproduction(reef:Reef, f_asexual:float):
    # Sort the coral in ascending order of fitness
    reef.sort_by_fitness()

    # Duplicate the best corals in the reef with a fraction 'f_asexual'
    corals_in_reef = [c for c in reef.corals_list if c is not None]
    asexual_set = corals_in_reef[: int(f_asexual * len([coral for coral in reef.corals_list if coral is not None]))] 

    return np.array(asexual_set) 

In [27]:
def reproduce_reef(reef:Reef, f_broadcast:float):
  pool = np.empty(shape=0, dtype=Individual) # pool of candidates to select from
  reproduction_candidates = [c for c in reef if c is not None]

  # Split reproduction sets
  [broadcast_set, brooding_set] = split_reef_candidates(reproduction_candidates, f_broadcast)

  # Broadcast    
  if len(broadcast_set) > 0: # If broadcast set is not empty, crossover
    while len(broadcast_set) > 0:
      # Select random father and a mother to crossover
      [father_candidate, mother_candidate] = np.random.choice(broadcast_set, 2, replace=False)
      offsprings = external_reproduction(father = father_candidate, 
                                          mother =  mother_candidate)

      # Add offspring to the pool
      pool = np.append(pool,offsprings[0]) # Two parents reproduce only one coral larva
      
      # Remove fathers of the reproduction list
      broadcast_set.remove(father_candidate)
      broadcast_set.remove(mother_candidate)
  
  #  Brooding
  if len(brooding_set) > 0: # If broadcast set is not empty, mutate every individual
    for c in brooding_set:
      internal_reproduction(c)
      pool = np.append(pool, c) # Add the mutated coral to the pool
  
  return pool

    

In [28]:
def predation(reef:Reef, f_depredation:float, depredation_probability:float):
  # Sort the coral in ascending order of fitness
  reef.sort_by_fitness()

  occupied_corals_in_reef = np.empty(shape=0, dtype=Individual) 
  empty_corals_in_reef = np.empty(shape=0, dtype=Individual)

  for coral in reef.corals_list:
    if coral is not None:
      occupied_corals_in_reef = np.append(occupied_corals_in_reef, coral)
      #print(f"Coral {coral.chromosome} has fitness {coral.fitness}")
    else:
      empty_corals_in_reef = np.append(empty_corals_in_reef, coral)
      #print("None")

  predation_number = int(f_depredation * len(occupied_corals_in_reef))
  
  for i in range(1, predation_number+1):
    # Remove the last corals in reef.corals_list if it is not None and if the depression probability is met
    if np.random.uniform() < depredation_probability:  
      #print(f"Deleting coral {occupied_corals_in_reef[-i].fitness}")
      occupied_corals_in_reef = np.delete(occupied_corals_in_reef, -i)
      empty_corals_in_reef = np.append(empty_corals_in_reef, None)

  reef.corals_list = np.append(occupied_corals_in_reef, empty_corals_in_reef)
  return reef

In [29]:
def cro(reef:Reef, max_generations: int, OPTIMAL_D:int, OPTIMAL_C:int, trainX:np.ndarray, trainY:np.ndarray, testX:np.ndarray, testY: np.ndarray):
  # Evaluate reef at the beginning
  compute_reef_fitness(reef, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

  t = 0
  while t < max_generations:
    # Update variables
    depredation_probability = 0.2 # predation probability
    f_broadcast = np.random.uniform() # broadcast fraction
    f_asexual = np.random.uniform() # asexual fraction

    f_depredation = f_asexual 
    if (f_depredation + f_asexual > 1):
      f_depredation = f_asexual = 0.5

    # Reproduction
    auxiliar_reef = np.copy(reef.corals_list) # Copy reef to avoid changing the original individuals
    pool = reproduce_reef(auxiliar_reef, f_broadcast)

    # Larvae settings
    reef = larvae_setting(pool, reef, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

    # Asexual reproduction
    asexual_set = asexual_reproduction(reef, f_asexual)

    reef.corals_list = np.append(reef.corals_list, asexual_set) # Add asexual set to the reef

    # Predation
    reef = predation(reef, f_depredation, depredation_probability)
        
    # Fitness Evaluation
    compute_reef_fitness(reef, OPTIMAL_D, OPTIMAL_C, trainX, trainY, testX, testY)

    # Increase step
    t += 1
  #end while

  reef.sort_by_fitness() # Final sort
  reef.best_coral = reef.corals_list[0] # Best coral is the first one in the final sorted reef
  return reef

# MAIN


In [30]:
# HYPERPARAMETERS FOR RUNNING SCRIPTS
# GENERAL
# K - declared already as the number of features
OPTIMAL_D = 100
OPTIMAL_C = 0.1

MAX_GENERATIONS = 10
POPULATION_SIZE = 5

# GA
crossover_prob = 0.65 # This value and mutation_prob are randomly set at Reproduce Population function
mutation_prob = 0.15

# PSO
w_max = 1 # inertia weight coefficient (max)
w_min = 0.1 # inertia weight coefficient (min)
c1 = 1.96 # cognitive coefficient
c2 = 1.96 # social coefficient

# CRO
f_broadcast = 0.5 # broadcast fraction
f_brooding = 1 - f_broadcast # brooding fraction
f_asexual = 0.2 # asexual fraction
f_depredation = 0.2 # depredation fraction
depredation_probability = 0.01 # predation probability
rho0 = 0.6 # free/occupied rate
#eta = 2 # larvae's attempts


In [31]:
# TESTING AREA - DO NOT TOUCH
CSV_PATH = "E:/Perfil/OneDrive/Escritorio/MrRobot/IITV/4/TFG/data-Vito-PC/"
CSV_NAME = "parkinsons.csv"

print(f"Executing ELM with dataset: {CSV_NAME}")

# Read CSV
df = pd.read_csv(CSV_PATH + CSV_NAME, sep=" ", header=None)
end = df.shape[1]

# - - - DATAFRAME CONSTANTS - - - #
X = df.iloc[:, 0:end-1].values
Y = df.iloc[:, end-1].values

J = len(np.unique(Y)) # Number of classes
N,K = X.shape[0],X.shape[1]  # N = number of samples, K = number of features

# - - - DATAFRAME PARTITIONS - - - #
X_scaled = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0)) # Scaling X (min-max normalization)
X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42) # First partition with 20% test data
X_trainVal, X_testVal, Y_trainVal, Y_testVal = train_test_split(X_train, Y_train, test_size=0.2, random_state=42) # Second partition with 20% validation data

# GA
ga_start_execution_time = time.time()
# Create initial population
population = Population.create_population(POPULATION_SIZE, K, J)

for i in population.genes_list:
    print(i)

lastGeneration = ga(population, MAX_GENERATIONS, OPTIMAL_D, OPTIMAL_C, X_trainVal, Y_trainVal, X_testVal, Y_testVal)
GA_CCR = train_model_and_output_results(best_weigths = lastGeneration.best_gene.weights, 
                                        best_chromosome = lastGeneration.best_gene.chromosome, 
                                        D = OPTIMAL_D,
                                        C = OPTIMAL_C, 
                                        trainingX = X_train,
                                        trainingY = Y_train,
                                        testX = X_test,
                                        testY = Y_test)
ga_execution_time = time.time() - ga_start_execution_time

# PSO
'''
pso_start_execution_time = time.time()
first_swarm = create_swarm(POPULATION_SIZE, K, OPTIMAL_D)  
final_swarm = pso(first_swarm, MAX_GENERATIONS, w_max, w_min, c1, c2, OPTIMAL_D, OPTIMAL_C, X_trainVal, Y_trainVal, X_testVal, Y_testVal)
PSO_CCR = train_model_and_output_results(best_weigths = final_swarm.global_best_weights, 
                                        best_chromosome = final_swarm.global_best_chromosome, 
                                        D = OPTIMAL_D,
                                        C = OPTIMAL_C, 
                                        trainingX = X_train,
                                        trainingY = Y_train,
                                        testX = X_test,
                                        testY = Y_test)
pso_execution_time = time.time() - pso_start_execution_time


# CRO
cro_start_execution_time = time.time()
reefFirstGeneration = create_reef(POPULATION_SIZE, rho0, K, OPTIMAL_D)
last_reef = cro(reefFirstGeneration, MAX_GENERATIONS, OPTIMAL_D, OPTIMAL_C, X_trainVal, Y_trainVal, X_testVal, Y_testVal)
CRO_CCR = train_model_and_output_results(best_weigths = last_reef.best_coral.weights, 
                                        best_chromosome = last_reef.best_coral.chromosome, 
                                        D = OPTIMAL_D,
                                        C = OPTIMAL_C, 
                                        trainingX = X_train,
                                        trainingY = Y_train,
                                        testX = X_test,
                                        testY = Y_test)
cro_execution_time = time.time() - cro_start_execution_time
'''

Executing ELM with dataset: parkinsons.csv

    Individual:
        Feature-selection: [1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 0 0 0 0]
        Weights: [[-0.20027806 -0.90666867]
 [ 0.94751104 -0.53445732]
 [-0.81878713  0.23677202]
 [-0.23507602  0.96646177]
 [-0.06647421  0.71988081]
 [ 0.36061508 -0.0990015 ]
 [-0.97347008  0.88440351]
 [ 0.12657644 -0.22916699]
 [-0.9680675  -0.53821235]
 [-0.51794907  0.36652704]
 [ 0.21999332  0.66638982]
 [-0.65327069 -0.21787878]
 [-0.63552782  0.51072282]
 [-0.14968825 -0.58411667]
 [ 0.13540066 -0.93737342]
 [ 0.68456955 -0.10049173]
 [-0.20969953  0.85331773]
 [ 0.45454399 -0.34691846]
 [ 0.14088795  0.04166852]
 [ 0.92234405  0.6890677 ]
 [ 0.49464022  0.07938426]]
        Bias: [[0.17350233 0.93051061]]
        Fitness: inf
    

    Individual:
        Feature-selection: [1 1 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 1]
        Weights: [[ 0.45801434  0.54254069]
 [-0.8519107  -0.28306854]
 [-0.76826188  0.72620685]
 [ 0.24659625 -0.33820395]
 [-0.

TypeError: __init__() missing 1 required positional argument: 'bias'

In [ ]:
'''
if __name__ == "__main__":
    PORTATIL_DEV = False

    if PORTATIL_DEV:
        CSV_PATH = "C:/Users/david/OneDrive/Escritorio/MrRobot/IITV/4/TFG/data-Vito-PC/"
    else:
        CSV_PATH = "E:/Perfil/OneDrive/Escritorio/MrRobot/IITV/4/TFG/data-Vito-PC/"
        
    data_directory = os.listdir(CSV_PATH)

    results_df = pd.DataFrame(columns=['Dataset','Generations', 'Population_size', 'GA', 'PSO', 'CRO', 'GA_time', 'PSO_time', 'CRO_time', 'D', 'C'])

    for index,csv in enumerate(data_directory):
        if (csv == "HandWriting.csv" or 
            csv == 'image-segmentation.csv' or 
            csv == 'ionosphere.csv' or 
            csv == "optical-recognition-handwritten-digits.csv" or
            csv == "seismic-bumps.csv"):
            continue
        
        print(f"Executing ELM with dataset: {csv}, {index+1} out of {len(data_directory)}")
        # Read CSV
        df = pd.read_csv(CSV_PATH + csv, sep=" ", header=None)
        end = df.shape[1]

        # - - - DATAFRAME CONSTANTS - - - #
        X = df.iloc[:, 0:end-1].values
        Y = df.iloc[:, end-1].values
        
        J = len(np.unique(Y)) # Number of classes
        N,K = X.shape[0],X.shape[1]  # N = number of samples, K = number of features

        # - - - DATAFRAME PARTITIONS - - - #
        X_scaled = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0)) # Scaling X (min-max normalization)
        X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42) # First partition with 20% test data
        X_trainVal, X_testVal, Y_trainVal, Y_testVal = train_test_split(X_train, Y_train, test_size=0.2, random_state=42) # Second partition with 20% validation data

        # GA
        ga_start_execution_time = time.time()
        first_population = create_population(POPULATION_SIZE, K, OPTIMAL_D)
        lastGeneration = ga(first_population, MAX_GENERATIONS, OPTIMAL_D, OPTIMAL_C, X_trainVal, Y_trainVal, X_testVal, Y_testVal)
        GA_CCR = train_model_and_output_results(best_weigths = lastGeneration.best_gene.weights, 
                                                best_chromosome = lastGeneration.best_gene.chromosome, 
                                                D = OPTIMAL_D,
                                                C = OPTIMAL_C, 
                                                trainingX = X_train,
                                                trainingY = Y_train,
                                                testX = X_test,
                                                testY = Y_test)
        ga_execution_time = time.time() - ga_start_execution_time

        # PSO
        pso_start_execution_time = time.time()
        first_swarm = create_swarm(POPULATION_SIZE, K, OPTIMAL_D)  
        final_swarm = pso(first_swarm, MAX_GENERATIONS, w_max, w_min, c1, c2, OPTIMAL_D, OPTIMAL_C, X_trainVal, Y_trainVal, X_testVal, Y_testVal)
        PSO_CCR = train_model_and_output_results(best_weigths = final_swarm.global_best_weights, 
                                                best_chromosome = final_swarm.global_best_chromosome, 
                                                D = OPTIMAL_D,
                                                C = OPTIMAL_C, 
                                                trainingX = X_train,
                                                trainingY = Y_train,
                                                testX = X_test,
                                                testY = Y_test)
        pso_execution_time = time.time() - pso_start_execution_time
        

        # CRO
        cro_start_execution_time = time.time()
        reefFirstGeneration = create_reef(POPULATION_SIZE, rho0, K, OPTIMAL_D)
        last_reef = cro(reefFirstGeneration, MAX_GENERATIONS, OPTIMAL_D, OPTIMAL_C, X_trainVal, Y_trainVal, X_testVal, Y_testVal)
        CRO_CCR = train_model_and_output_results(best_weigths = last_reef.best_coral.weights, 
                                                best_chromosome = last_reef.best_coral.chromosome, 
                                                D = OPTIMAL_D,
                                                C = OPTIMAL_C, 
                                                trainingX = X_train,
                                                trainingY = Y_train,
                                                testX = X_test,
                                                testY = Y_test)
        cro_execution_time = time.time() - cro_start_execution_time

        csv_results = pd.DataFrame({'Dataset': csv,
                                    'Generations': MAX_GENERATIONS,
                                    'Population_size': POPULATION_SIZE,
                                    'GA': GA_CCR,
                                    'PSO': PSO_CCR,
                                    'CRO': CRO_CCR,
                                    'D': OPTIMAL_D, 
                                    'C': OPTIMAL_C, 
                                    'GA_time': ga_execution_time, 
                                    'PSO_time': pso_execution_time,
                                    'CRO_time': cro_execution_time }, index=[0])

        # Concat results of the experiments to the results dataframe
        results_df = pd.concat([results_df, csv_results], ignore_index=True)

        # Debug
        #print("GA CCR:", GA_CCR, "\nError:", 100-GA_CCR)
        #print("GA execution time: ", time.time() - ga_start_execution_time, "sec")
        #print(f"Best solution found in PSO: {final_swarm.best_gene}")
        #print(f"Maximum weight value found: {final_swarm.best_gene.weights.max()} - Minimum weight value found: {final_swarm.best_gene.weights.min()}")
        #print(f"Maximum velocities value found: {final_swarm.best_gene.weights_velocity.max()} - Minimum velocities value found: {final_swarm.best_gene.weights_velocity.min()}")
        #print("PSO CCR:", PSO_CCR, "\nError:", 100-PSO_CCR)
        #print("PSO execution time: ", time.time() - pso_start_execution_time, "sec")
        #print("CRO CCR:", CRO_CCR, "\nError:", 100-CRO_CCR)
        #print("CRO execution time: ", time.time() - cro_start_execution_time, "sec")
            
    # end for
    results_df.to_excel("results.xlsx", sheet_name='ELM_' + str(date.today()) + time.strftime("_%H_%M_%S") , index=False)
    print("Results saved to results.xlsx!")
'''
